## Alcholic Anonymous News articles extraction from Deccan Chronicle. Data Extraction of following parameters
- Headline
- Description
- Author
- Published_Date
- News
- URL
- Keywords
- Summary

### Importing the necessary Libraries

In [1]:
from selenium.webdriver.chrome.options import Options # enables options in web browser
from selenium import webdriver # web-based automation tool for Python
from newspaper import Article # Article scraping & curation
from bs4 import BeautifulSoup # Python library for pulling data out of HTML and XML files
from requests import get # standard for making HTTP requests in Python
import pandas as pd # library written for data manipulation and analysis
import sys, time #  System-specific parameters and functions.

### Creating Empty lists for Alcholic Anonymous News Articles parameters data to be extracted

In [2]:
headlines, descriptions, dates, authors, news, keywords, summaries, urls = [], [], [], [], [], [], [], []

### Finding the total no.of.pages by total no.of articles from google search results¶

In [3]:
keyword = 'Alcoholics Anonymous site:www.deccanchronicle.com'

url = 'https://www.google.com/search?q=' + '+'.join(keyword.split())

soup = BeautifulSoup(get(url).text, 'lxml')
try:
    # Extracts the digits if it the resulted number without comma ','. eg: About 680 results (0.23 seconds)
    max_pages = round([int(s) for s in soup.select_one('div#resultStats').text.split() if s.isdigit()][0]/10)
    max_pages = max_pages + 1
except:
    # Extracts the digits if it the resulted number without comma ','. eg: About 1,080 results (0.23 seconds)
    max_pages = round(int(''.join(i for i in soup.select_one('div#resultStats').text if i.isdigit()))/10)
    max_pages = max_pages + 1

### Iterates max_pages value through while loop. Scraping the Articles urls

In [4]:
options = Options()
options.headless = True
browser = webdriver.Chrome(options=options)
browser.get(url)

index = 0

while True:
    try:
        index +=1
        page = browser.page_source
        soup = BeautifulSoup(page, 'lxml')
        linky = [soup.select('.r')[i].a['href'] for i in range(len(soup.select('.r')))]
        urls.extend(linky)
        if index == max_pages:
            break
        browser.find_element_by_xpath('//*[@id="pnnext"]/span[2]').click()
        time.sleep(2)
        sys.stdout.write('\r' + str(index) + ' : ' + str(max_pages) + '\r')
        sys.stdout.flush()
    except:
        pass
    
browser.quit()

### To remove duplicates urls entries in the list by executing below line

In [5]:
urls = list(dict.fromkeys(urls))
print(len(urls), type(urls))

29 <class 'list'>


### Iterates urls through for loop. Scraping the Articles using BeautifulSoup4 which extracts above parameters

In [6]:
%%time
for index, url in enumerate(urls):
    try:
        # Parse the url to NewsPlease 
        soup = BeautifulSoup(get(url).text, 'lxml')
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()
        
        # Extracts the Headlines
        try:
            try:
                headlines.append(article.title.strip())
            except:
                headlines.append(soup.select_one('meta[property="og:title"]')['content'].strip())
        except:
            headlines.append(None)
            
        # Extracts the Descriptions    
        try:
            try:
                descriptions.append(soup.select_one('meta[property="og:description"]')['content'].strip().replace('\n', ' '))
            except:
                descriptions.append(article.meta_description.strip())
        except:
            descriptions.append(None)
            
        # Extracts the Authors
        try:
            try:
                authors.append(soup.select_one('meta[name="author"]')['content'].strip())
            except:
                authors.append(article.authors.strip())
        except:
            authors.append(None)
        
        # Extracts the published dates
        try:
            try:
                dates.append(soup.select_one('meta[property="article:published_time"]')['content'].strip())
            except:
                dates.append(str(article.publish_date))
        except:
            dates.append(None)
            
        # Extracts the news articles
        try:
            try:
                news.append(soup.select_one('.storyText').text.replace('\n', '').strip())
            except:
                news.append(article.text.strip())
        except:
            news.append(None)
            
        # Extracts Keywords and Summaries    
        try:
            keywords.append(article.keywords)
            summaries.append(article.summary)
        except:
            keywords.append(None)
            summaries.append(None)
            
    except:
        headlines.append(None)
        descriptions.append(None)
        authors.append(None)
        dates.append(None)
        news.append(None)
        keywords.append(None)
        summaries.append(None)

    sys.stdout.write('\r' + str(index) + ' : ' + str(url) + '\r')
    sys.stdout.flush()

C:\ProgramData\Anaconda3\lib\site-packages\dateutil\parser\_parser.py:1204: UnknownTimezoneWarning: tzname IST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


Wall time: 1min 59sccanchronicle.com/nation/current-affairs/240817/ram-rahim-singh-confirms-hell-appear-in-court-for-verdict-tomorrow.html9


### Checking Array Length of each list to create DataFrame

In [7]:
print(len(headlines), len(descriptions), len(authors), len(dates), len(news), len(keywords), len(summaries), len(urls))

29 29 29 29 29 29 29 29


### Creating a csv file after checking array length and droping the missing values from the dataset

In [8]:
tbl = pd.DataFrame({'Headlines' : headlines,
                    'Descriptions' : descriptions,
                    'Authors' : authors,
                    'Published_Dates' : dates, 
                    'Articles' : news,
                    'Keywords' : keywords,
                    'Summaries' : summaries,})

tbl.dropna()
tbl.to_csv('Deccan_Chronicle.csv', index=False)
tbl.head()

,Headlines,Descriptions,Authors,Published_Dates,Articles,Keywords,Summaries
0,Alcoholics Anonymous,Alcoholics Anonymous,None,None,Alcoholics Anonymous: The helping hand for add...,"[alcoholics, men, share, ngo, group, fellowshi...",Alcoholics Anonymous: The helping hand for add...
1,Alcoholics Anonymous: The helping hand for add...,This group is not a rehabilitation centre or a...,None,"May 3, 2016, 3:24 am IST",What begins as social drinking turns into alco...,"[alcoholics, addicts, alcoholic, group, alcoho...",What begins as social drinking turns into alco...
2,"We are addicted to the absence of absence, say...",Actor-director Pooja Bhatt talks about dealing...,None,"Apr 1, 2017, 12:23 am IST","Power, determination and the unabashed guts to...","[bhatt, things, pooja, day, life, addicted, ch...","Power, determination and the unabashed guts to..."
3,Helping alcoholics kick the bottle!,Today Harsha is a happily married man and he c...,None,"Aug 28, 2017, 1:23 am IST",BENGALURU: Despite measures being taken by sev...,"[alcoholics, volunteers, members, alcoholic, a...",BENGALURU: Despite measures being taken by sev...
4,"AA reaches out to schools now, rural men in TN...",Consumption of alcohol is also high in Tamil N...,None,"Aug 1, 2018, 6:15 am IST",Thanjavur: It will be startling to note that t...,"[drinking, men, alcohol, consume, urban, reach...",53.9 per cent of men and 8.8 per cent of women...
